# How to create a functional pipeline for file output with `toolz.pipe`? 

**Tags:** [python],  [lambda], [toolz], [functional-programming]


A curiously recurring pattern in python, is the need to call the python functions `open`, `write` and `close` to output text to a file. 

How can `toolz.pipe` be used to compose these functions into a resuable functional pipline?


## Imperative implementation

Let's start with the following imperative implementation.

In [1]:
import json

DATA = dict(key="json key", value="json value")
TEXT = json.dumps(DATA)
FILENAME1 = "filename1.json"

io = open(FILENAME1, mode='w')                                  # line 1: open
chars = io.write(TEXT)                                          # line 2: write
io.close()                                                      # line 3: close
print(f"Wrote the file `{FILENAME1}` with {chars} characters.") # line 4: print


Wrote the file `filename1.json` with 42 characters.


This is the way that file output it is commonly done in python. Instead of having to repeat these three function calls for every file that we wish to output, it would be convenient to be able to chain these three functons together into a single reusable pipeline for writing output files. 

## Object-oriented implementation

This can be simply and easily implemented with object-oriented programing. The object is used to manage the state of the intermediate values returned from each `open` and `write` and `close` function call. 


In [2]:
import _io
from dataclasses import dataclass

@dataclass
class File:
    """File."""

    name: str
    io: _io.TextIOWrapper = None
    chars: int = 0

    def write(self, text):
        """Write file."""
        self.io = open(self.name, mode='w')
        self.chars = self.io.write(text)
        self.io.close()
        return self

    def show(self):
        """Show file."""
        print(f"Wrote the file `{self.name}` with {self.chars} characters.")



The object oriented implementaiton can be simply and consicely reused to output any data to a file anywhere in your program as follows;


In [3]:
FILENAME2 = "filename2.json"

File(FILENAME2).write(TEXT).show()

Wrote the file `filename2.json` with 42 characters.



This leads to my actual question;

* What should the equivalent functional programming implementatiom look like?
 


## Functional implementation

After having recently read many articles about the many benefits of functional programming, I tired to combine the `open`, `write` and `close` functions into a single functional pipeline that can be reused without the need to name the intermediate variables storing the state. A secondary benefit would be that the intermediate states can be made immutable and safely encapsulated using a `dataclass`. 

Using the `toolz.pipe` function, I came up with this implementaiton which produces the same output and side effect (writing file) as the imperative implementation. 


In [4]:
from dataclasses import replace
from toolz import do, pipe

@dataclass(frozen=True)
class File:
    """File."""

    name: str
    io: _io.TextIOWrapper = None
    chars: int = 0

pipe(File("filename3.json"),
    lambda file: replace(file, io=open(file.name, mode='w')),                   # line 1: open
    lambda file: replace(file, chars=file.io.write(TEXT)),                      # line 2: write
    lambda file: do(lambda file: file.io.close(), file),                        # line 3: close
    lambda file: f"Wrote the file `{file.name}` with {file.chars} characters.") # line 4: print

'Wrote the file `filename3.json` with 42 characters.'

To my surprise, this funcitonal implementation is not nearly as concise or as simple as the imperative and object orented implementations. I need to understand why this is the case; especially since programming in a functional style promises to deliver so many benefits.

This leads me to ask the following specific questions;

* Is there a better way to implement this using `toolz.pipe`?

    For example;
    - Not using so many lambdas. 
    - Not requiring a `dataclass` to manage the stateful intermediate variables. 

* Are there some other `toolz` library functions that could be used to improve and simplify the implementation (e.g. `curry`, `compose_left`)? 
* Is the `toolz.pipe` function being used correctly and as it was intended to be used?


In [5]:
from toolz import compose_left

compose_left()

ImportError: cannot import name 'compose_left'

In [6]:
from toolz.curried import pipe 

In [7]:
compose_left(lambda x: x**2, lambda x: f"result = {x}")

NameError: name 'compose_left' is not defined